# 0.988 --> 0.99228 or More

My challenge to myself is to beat my old score. I will be trying different CNN. I expanded the dimensions to get better result in return. I am going to explain the process to it...

In [1]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow 

import scipy
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


# Set the seed value for experiment reproduci.bility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')

Num GPUs Available:  1


2022-08-13 17:02:21.740331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-13 17:02:21.937855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-13 17:02:21.941894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Firstly, I am importing the libraries that I will use. Then, I am deciding the shape of data and transforming it to (28,28,1).

In [2]:
df=pd.read_csv('../input/digit-recognizer/train.csv')

X=df.drop('label',axis=1)
Y=df.label
Y=pd.get_dummies(Y,prefix='Num')

In [3]:
X=X/255

X=X.values.reshape(-1,28,28,1)

From sklearn library, I am splitting the train and val values for our training and validation process.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

validation_data=X_val,y_val

# Model

## First CNN

In [5]:
#experiment with Convolutional neural net
model = keras.models.Sequential([
keras.layers.RandomRotation(0.1),
    
keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape = (28,28,1)),
keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Conv2D(128, (2, 2), activation='relu'),
keras.layers.Conv2D(128, (2, 2), activation='relu'),
keras.layers.Conv2D(128, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Flatten(),
keras.layers.Dropout(0.3),
keras.layers.Dense(512, activation='relu'),
keras.layers.Dropout(0.25),
keras.layers.Dense(10, activation ='softmax')])

2022-08-13 17:02:28.035121: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-13 17:02:28.035690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-13 17:02:28.039901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-13 17:02:28.044275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

## Compile/Training 

An ROC curve (receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. AUC stands for "Area under the ROC Curve." That is, AUC measures the entire two-dimensional area underneath the entire ROC curve (think integral calculus) from (0,0) to (1,1).

I am using AUC and acc as metrics and I want to check the results. 

EarlyStopping is a choice that I generally use to control the model output. Monitoring val_acc is a choice again. I got better model with that.

I generally use LearningRateScheduler. I observe better results.

It is vital to save the best model for submitting your best score...

In [6]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_acc',
                                            patience=7,
                                            restore_best_weights=True)


def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = keras.callbacks.ModelCheckpoint("best_model.h5", monitor="val_acc",
                                                    save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

In [7]:
history=model.fit(X_train,y_train, epochs=40, validation_data=validation_data, callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler])

2022-08-13 17:02:32.211426: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2022-08-13 17:02:35.331427: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1050/1050 [==============================] - 22s 11ms/step - loss: 0.4661 - auc: 0.9839 - acc: 0.8579 - val_loss: 0.1973 - val_auc: 0.9935 - val_acc: 0.9521
Epoch 2/40
1050/1050 [==============================] - 12s 12ms/step - loss: 0.2041 - auc: 0.9948 - acc: 0.9406 - val_loss: 0.0899 - val_auc: 0.9984 - val_acc: 0.9746
Epoch 3/40
1050/1050 [==============================] - 11s 11ms/step - loss: 0.1577 - auc: 0.9961 - acc: 0.9565 - val_loss: 0.1839 - val_auc: 0.9968 - val_acc: 0.9669
Epoch 4/40
1050/1050 [==============================] - 11s 11ms/step - loss: 0.1353 - auc: 0.9967 - acc: 0.9626 - val_loss: 0.0805 - val_auc: 0.9982 - val_acc: 0.9752
Epoch 5/40
1050/1050 [==============================] - 11s 11ms/step - loss: 0.1156 - auc: 0.9973 - acc: 0.9687 - val_loss: 0.1780 - val_auc: 0.9960 - val_acc: 0.9708
Epoch 6/40
1050/1050 [==============================] - 13s 12ms/step - loss: 0.1097 - auc: 0.9973 - acc: 0.9704 - val_loss: 2.7576 - val_auc: 0.9938 - val_acc: 0.9735
Epo

I am bringing the best model in and checking the loss,auc and accuracy values...

In [8]:
from tensorflow.keras.models import load_model
model=load_model(
    "best_model.h5", custom_objects=None, compile=True, options=None
)
loss, auc, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("AUC: ", auc)
print("Accuracy: ", accuracy)

263/263 [==============================] - 2s 5ms/step - loss: 0.0262 - auc: 0.9993 - acc: 0.9942
Loss:  0.026235835626721382
AUC:  0.9992613792419434
Accuracy:  0.9941666722297668


## Submission

In order to submit with the correct format I am using np.argmax. You may get 0 score if you are submitting with wrong format...

In [9]:
df_test=pd.read_csv('../input/digit-recognizer/test.csv')
df_test=df_test/255

df_test=df_test.values.reshape(-1,28,28,1)

y_pred=model.predict(df_test)


y_pred=pd.DataFrame(np.argmax(y_pred,axis=1),columns=['Label'])
submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
submission["Label"] = y_pred
submission.to_csv('submission.csv', index=False)